In [1]:
from KG.kg import ingest_Chunks, create_nodes, create_relationship, create_vector_index, embed_text
from KG.chunking import split_data_from_file
import KG.config as config
import json, os
graph = config.load_neo4j_graph()

c:\Users\leonu\Desktop\Dokumente\Research\Graph_RAG\KG\config.py:15: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [2]:
file_names = ["Battle_of_Waterloo", "Napoleon", "Talleyrand"]

for name in file_names:
    # load JSON file
    file = f"data/{name}.json"
    # chunking
    chunks = split_data_from_file(file)
    # assuming 'file' is a path to JSON file
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if name == "Battle_of_Waterloo":
        create_nodes(graph=graph, data=data, node_label="Event", node_name=name)
    else:
        create_nodes(graph=graph, data=data, node_label="Person", node_name=name)
    #ingest chunks
    ingest_Chunks(graph=graph, chunks=chunks, node_name=name, node_label='Chunk')


['General information', 'Reason', 'Combatant', 'Consequence']
Processing General information from data/Battle_of_Waterloo.json
	Split into 11 chunks
Processing Reason from data/Battle_of_Waterloo.json
	Split into 9 chunks
Processing Combatant from data/Battle_of_Waterloo.json
	Split into 48 chunks
Processing Consequence from data/Battle_of_Waterloo.json
	Split into 58 chunks
Creating ':Chunk' node for chunk ID Battle_of_Waterloo-General information-chunk0000
Creating ':Chunk' node for chunk ID Battle_of_Waterloo-General information-chunk0001
Creating ':Chunk' node for chunk ID Battle_of_Waterloo-General information-chunk0002
Creating ':Chunk' node for chunk ID Battle_of_Waterloo-General information-chunk0003
Creating ':Chunk' node for chunk ID Battle_of_Waterloo-General information-chunk0004
Creating ':Chunk' node for chunk ID Battle_of_Waterloo-General information-chunk0005
Creating ':Chunk' node for chunk ID Battle_of_Waterloo-General information-chunk0006
Creating ':Chunk' node for 

In [3]:
# Person with person
rel_person_person = """
MATCH (p1:Person), (p2:Person)
WHERE id(p1) < id(p2)
MERGE (p1)-[:RELATED_TO]->(p2)
MERGE (p2)-[:RELATED_TO]->(p1);

"""
# Section with chunk
rel_section_chunk = """ 
MATCH (s:Section), (c:Chunk)
WHERE s.type = c.source AND s.parent_name = c.node_name
MERGE (s)-[:HAS_CHUNK]->(c);

"""
# Person with chunk
rel_person_section = """
MATCH (p:Person), (s:Section)
WHERE p.name = s.parent_name
MERGE (p)-[:HAS_SECTION]->(s);

"""
# Event with section
rel_event_section = """
MATCH (e:Event), (s:Section)
WHERE e.name = s.parent_name
MERGE (e)-[:HAS_SECTION]->(s);

"""
# Person to event
rel_person_event = """
MATCH (p:Person), (e:Event)
MERGE (p)-[:RELATED_TO]->(e)
MERGE (e)-[:RELATED_TO]->(p);

"""

queries = [rel_section_chunk, rel_person_person, rel_person_event, rel_person_section, rel_event_section]

for query in queries:
    create_relationship(graph=graph, query=query)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function. ('id' has been replaced by 'elementId or consider using an application-generated id')} {position: line: 3, column: 7, offset: 38} for query: '\nMATCH (p1:Person), (p2:Person)\nWHERE id(p1) < id(p2)\nMERGE (p1)-[:RELATED_TO]->(p2)\nMERGE (p2)-[:RELATED_TO]->(p1);\n\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function. ('id' has been replaced by 'elementId or consider using an application-generated id')} {position: line: 3, column: 16, offset: 47} for query: '\nMATCH (p1:Person), (p2:Person

In [4]:
# Create Vector Index
create_vector_index(graph=graph, index_name='Chunk')

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = config.SERVICE_ACCOUNT_KEY_PATH
os.environ["GOOGLE_CLOUD_PROJECT"] = config.G_PROJECT_NAME
os.environ["GOOGLE_CLOUD_LOCATION"] = config.G_PROJECT_LOCATION

# Embedding chunks with empyt vectors
embed_text(
    graph=graph, 
    GOOGLE_CLOUD_PROJECT=os.environ["GOOGLE_CLOUD_PROJECT"], 
    GOOGLE_CLOUD_LOCATION=os.environ["GOOGLE_CLOUD_LOCATION"], 
    node_name='Chunk'
)

c:\Users\leonu\Desktop\Dokumente\Research\Graph_RAG\.venv\Lib\site-packages\vertexai\_model_garden\_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Starting embedding update...
Found 300 nodes without embeddings.


Embedding nodes: 100%|████████████████████████████████████████████| 300/300 [02:21<00:00,  2.12it/s]

Finished embedding update.
